In [ ]:
# Using a seed causes random number generators to always select the same number, reducing randomness 
# and increasing comparability of K fold results
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [ ]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch.nn as nn
# Define the model
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2) 
        self.linear3 = nn.Linear(H2, H3) 
        self.linear4 = nn.Linear(H3, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))
        x = self.linear4(x)
        return x



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm
import xlrd
import pandas as pd

# import the data
book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,9)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)

book = xlrd.open_workbook('prepareddatakfold.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_replicated = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7200)]

# standardise the data
scaler = StandardScaler()
scaler.fit(data_unreplicated)
standardised_data = scaler.transform(data_replicated)

x_train = standardised_data[:, 0:5]
y_train = standardised_data[:, 5:8]

# split data into folds
n_splits = 6
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))



In [ ]:
import copy
i = n_splits-1
batches = 50
learning_rate = 5e-4
loss_fn = torch.nn.MSELoss(reduction='sum')
# choose the dimensions of the grid search 
for q in range (5,25,5):
    for l in range (5,25,5):
        for m in range (3,15,3):
            # Define the model to be tested, create a copy of parameters to load at the beginning of each fold
            # so that each tested model with the same number of neurons has the same intial parameters 
            N, D_in, H1, H2, H3, D_out = batches, 5, q, l , m, 3
            model = Net(D_in, H1, H2, H3, D_out)
            model.train()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            init_state = copy.deepcopy(model.state_dict())
            init_state_opt = copy.deepcopy(optimizer.state_dict())
            av_opt_epoch=0
            av_opt_loss=0
            for i, (train_idx, valid_idx) in enumerate(splits):
                # split data into validation fold and test folds
                model.load_state_dict(init_state)
                optimizer.load_state_dict(init_state_opt)
                x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
                y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
                x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
                y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
                train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
                valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
                # put data into batches
                train_loader = torch.utils.data.DataLoader(train, batch_size=batches,
                                                           shuffle=True)
                valid_loader = torch.utils.data.DataLoader(valid, batch_size=batches,
                                                           shuffle=False)
                UP=0
                epoch=0
                k=0
                opt_loss=1000
                old_avg_val_loss=0
                
                # setup stopping conditon
                while UP<6 and epoch<300:
                    epoch+=1
                    k+=1
                    model.train()
                    # train the model
                    for x_batch, y_batch in train_loader:
                        y_pred = model(x_batch)
                        loss = loss_fn(y_pred, y_batch)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    model.eval()
                    avg_val_loss = 0.
                    # test against validation set
                    for x_batch, y_batch in valid_loader:
                        y_pred = model(x_batch).detach()
                        avg_val_loss += (loss_fn(y_pred, y_batch).item() / 
                                         (len(valid_loader.dataset)))
                    # test against stopping conditions
                    if k > 4:
                        if avg_val_loss> old_avg_val_loss:
                            UP+=1
                        if avg_val_loss<old_avg_val_loss:
                            UP=0
                        old_avg_val_loss=avg_val_loss
                        k=0
                    if avg_val_loss<opt_loss:
                        opt_loss = avg_val_loss
                        opt_epoch = epoch
                        
                # collate and print model results
                av_opt_epoch+=opt_epoch/n_splits
                av_opt_loss+=opt_loss/n_splits
            print(H1,H2,H3,av_opt_epoch,av_opt_loss)
         
